# Text processing with regular expressions #

In this lecture we will discuss extracting information from strings/pieces of text. By information we mean here a substring which satisfies certain specifications, e.g. it represents a number or valid email address. We will see that this task can sometimes get quickly out of hand - capturing all the corner cases will lead to code which maybe hard to read and maintain. This is where regular expression will enter the stage to save the day. 

## String methods
Recall the previously seen methods of `str` objects
```python
str.find
str.index
str.count
str.replace
```
which are suitable for working with specifications given by fixed characters. For example, a simple re-implementation of `os.path.splitext` could read

In [13]:
import os

def splitext(path):
    '''Naive split extension in path'''
    if path.count('.') == 0:
        return None
    # Unpack the list as list[:-1], list[-1]
    *noext, ext = path.split('.')
    return (''.join(noext), f'.{ext}')

print(splitext('foo_bar'), splitext('foo_bar'))
print(splitext('foo.bar'), os.path.splitext('foo.bar'))
print(splitext('foo.bar.txt'), os.path.splitext('foo.bar.txt'))

None None
('foo', '.bar') ('foo', '.bar')
('foobar', '.txt') ('foo.bar', '.txt')


### What is a number?
What if the specifications are not as easy to define? In the following we will attempt to write a fuction which returns a list of numbers contained in a string. Before finding many we shold be able to find just one.

In [24]:
# Our first definition motivated by integers
def only_digits(string):
    if set(string) <= {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}:
        return True
    return False

print(only_digits('123a'))
print(only_digits('123'))

# But what about negative numbers
def is_integer(string):
    if len(string) == 1:
        return only_digits(string)
    maybe_sign, rest = string[0], string[1:]
    if maybe_sign == '-' and only_digits(rest):
        return True
    
    return only_digits(string)
    
print(is_integer('-123'))

False
True
True


But what about `floats`, 0.234 and scientific formating 0.234E-04. We could probably handle all these after a while but here we will settle for a different approach: a number string is that string which can be converted to a number 

In [22]:
def is_number(string):
    '''It is a number if it can be converted'''
    try:
        float(string)
        return True
    except ValueError:
        return False
    
print(is_number(-3.43E-02))

True


So far we have considered strings that were either number or not. But what if the strings are more involved. For example let us consider some string encoding wind speed and temperature 'wind27temperature-32'. We don't expect 
```python
is_number('wind27temperature32')
```
to work but we could start building a solution on this function.

In [26]:
is_number('wind27temperature32')

False

In [30]:
def find_numbers(string):
    numbers = []
    first, n = 0, len(string)
    while first < n:
        maybe = string[first:first+1]
        # If we have a number, grow it as much as possible
        if is_number(maybe):
            last = first
            while last < n and is_number(string[first:last+1]):
                maybe = string[first:last+1]
                last += 1
            numbers.append(maybe)
            first = last
        else:
            first += 1
            
    return numbers
print(find_numbers('wind27temperature32'))
print(find_numbers('wind27temperature42.23423'))

['27', '32']
['27', '42.23423']


So this looks promising but what about negetive temperatures and that scientific formatting again?

In [201]:
print(find_numbers('wind27temperature-42.234E-23'))

['27', '42.234', '23']


Not quite what we want ... .The problem above was matching on the smallest strings so (-) is ignore because it is alone and we don't include 42.234E-23 because "42.234E" is `False` according to `is_number`. We are back to the drawing board.

In [202]:
def get_substrings(string):
    '''Generated substring of string order by descending ize'''
    substrings = []
    n = len(string)
    for subl in range(n-1, 0, -1):
        for i in range(n+1-subl):
            substrings.append(string[i:i+subl])
    return substrings


def find_numbers1(string):
    '''Match from largest'''
    if not string:
        return []

    if is_number(string):
        return [string]

    numbers = []
    for substring in get_substrings(string):
        # print(substring, is_number(substring))
        # For a match we clip the string and as for matches in the neighbors
        if is_number(substring):
            numbers.append(substring)
            # print('->', substring, string[:string.index(substring)], string[string.index(substring)+len(substring):])
            numbers.extend(find_numbers1(string[:string.index(substring)]))
            numbers.extend(find_numbers1(string[string.index(substring)+len(substring):]))
            break
    return numbers
            
print(find_numbers1('wind27temperature-42.234E-23'))
print(find_numbers1('wind27temperature-42.234E-23 wind27temperature-42.234e123'))

['-42.234E-23', '27']
['-42.234E-23 ', '27', '-42.234e123', '27']


This seems to work but (while it may have been fun to write this code) you should have a feeling that [There must be a better way](https://fullstackfeed.com/there-must-be-a-better-way-raymond-hettinger-python/). 

# Regular expressions
- sequence of characters that defines a search pattern
- the pattern is parsed/interpretted in a special way (cf. programming language)
- they are not unique to Python, see [RegEx](https://regex101.com/)
- in Python the functionality for RegEx is provided in the [re](https://docs.python.org/3/library/re.html) module 

Here we will demonstrate the basics of regex syntax using the top-level function `re.search` and return to more of the module's functions later
```python
import re
re.search(pattern:str, string:str, [flags]) -> re.Match or None
```
Match specifies the part of string where pattern was matched

In [42]:
import re
print(re.search('tractor', 'attractor'))
print(re.search('tractor', 'Attractor'))
print(re.search('tractor', 'ATTractor', re.IGNORECASE))  # re.IGNORECASE is our first flag

<re.Match object; span=(2, 9), match='tractor'>
<re.Match object; span=(2, 9), match='tractor'>
<re.Match object; span=(2, 9), match='Tractor'>


In [203]:
pattern, string = 'tractor', 'Attractor'
match = re.search(pattern, string)
print(string[match.start():match.end()])

tractor


And so we see that "simple words" specify a pattern that simply matches itself. 
However, we could already do that with `str` methods. To specify more intesting 
patterns RegEx use special syntax.

## Metacharacters
The following characters have a special meaning in defining the pattern
```
. ^ $ * + ? { } [ ] \ | ( )
```
When one wants to match them literally they need to be escaped

In [49]:
print(re.search('^', '2^6'))       # We have a match ?!
print(re.search('\^', '2^6'))      # Here as expected

<re.Match object; span=(0, 0), match=''>
<re.Match object; span=(1, 2), match='^'>


Square brackets delineate a character class. Characters can be listed individually ...

In [55]:
print(re.search('[ea]', 'long lean'))  #  Match at e or a
print(re.search('[oea]', 'long lean'))  #  Why did we stop here
print(re.search('[ea]', 'long last lean'))

<re.Match object; span=(6, 7), match='e'>
<re.Match object; span=(1, 2), match='o'>
<re.Match object; span=(6, 7), match='a'>


... or given as ranges, which can be combined, ...

In [204]:
print(re.search('[a-z]', 'life'))
print(re.search('[a-zA-Z0-9]', '____7up'))  # Walk though here

<re.Match object; span=(0, 1), match='l'>
<re.Match object; span=(4, 5), match='7'>


and complemented

In [206]:
print(re.search('[^a]', 'abc'))
print(re.search('[^a]', 'bc'))
print(re.search('[^a-z]', 'life'))
print(re.search('[^a-z]', 'up7'))

<re.Match object; span=(1, 2), match='b'>
<re.Match object; span=(0, 1), match='b'>
None
<re.Match object; span=(2, 3), match='7'>


Note that metacharacters (except backslash) are not active inside a character class. 
For example we have seen a match in `re.search('^', '2^6')` because `^` means (normally) the anchor for start of a string. 

Backslash `\` is another another metacharacter. We recall that `\` has 
a special meaning in Python strings, e.g. '\n', '\t' are not simply strings of 2 characters but represent carriage return/newline 
and tab respecively. To supress this beviour we will use raw string `r"string"`

In [62]:
(len('\n'), len(r'\n'))

(1, 2)

Speical sequences beginning with `\`

- `\d` : matches a single character that is a digit, `[0-9]`
- `\w` : matches a word character, `[a-zA-Z0-9_]`  (Note the underscore)
- `\s` : matches a whitespace character, ` [ \t\n\r\f\v]`

And their negations

- `\D` : matches a single character that is a not digit, `[^0-9]`
- `\W` : matches a non-word character, `[^a-zA-Z0-9_]`
- `\S` : matches a non-whitespace character, ` [^ \t\n\r\f\v]`

Note that these preserve their meaning inside character classes

In [207]:
re.search('[\d+.\d+]', 'asasd 323.23')   # Walk through

<re.Match object; span=(6, 7), match='3'>

Other special characters
- `.`  matches any character except a newline
- `|` the OR operator

In [67]:
print(re.search('.', '\n'))
print(re.search('.', ' '))
print(re.search('.', ' vv'))

print(re.search('a|b', 'xerox'))
print(re.search('a|b', 'vibe'))

None
<re.Match object; span=(0, 1), match=' '>
<re.Match object; span=(0, 1), match=' '>
None
<re.Match object; span=(2, 3), match='b'>


## Anchors

Characters for specifing position or RE within the string 
- `^` match at the beginning of the string (but `re.MULTILINE`)
- `\A` matches only at the start of the string
- `$` match at the end of string (but `re.MULTILINE`)
- `\Z` matches only at the start of the string
- `\b` word boundary
- `\B` not word boundary

In [208]:
print(re.search('^a', 'ball'))
print(re.search('^a', 'ball\nall', re.MULTILINE))
print(re.search('a', 'ball'))
# NOTE: '\b' is the bell character hence r'...'
print(re.search(r'\ball\b', 'ballistic'))
print(re.search(r'\ball\b', 'all'))
print(re.search(r'\ball\b', 'allabama'))
print(re.search(r'\ball\B', 'allabama'))

None
<re.Match object; span=(5, 6), match='a'>
<re.Match object; span=(1, 2), match='a'>
None
<re.Match object; span=(0, 3), match='all'>
None
<re.Match object; span=(0, 3), match='all'>


## Repeating patterns

**Quantifiers**

The characters `*`, `+`, `?` and `{}` are reserved as quantifiers.

For example, 
- `*` 0 or more occurances of previous character/RE
- `+` 1 or more occurances or previous character/RE
- `?` 0 or 1 occurances of previous character/RE
- `{n}` exactly `n` occurances of previous character/RE
- `{n,}` at least `n` occurances
- `{n, m}` n to m (included) occurances 

__Question__: 
1. What is the equivalent of `{0, }`
2. What is the equivalent of `{1, }`

In [105]:
print(re.search('i*', 'team'))
print(re.search('i+', 'team'))
print(re.search('i+', 'team spirit'))
print(re.search(r'\B(i{1})\B', 'spxirit'))

<re.Match object; span=(0, 0), match=''>
None
<re.Match object; span=(7, 8), match='i'>
<re.Match object; span=(3, 4), match='i'>


In [107]:
# With classes
print(re.search('[aeiou]+', 'team'))
print(re.search('[aeuio]+', 'lynx'))
print(re.search('[\d]+', 'SR 71'))

<re.Match object; span=(1, 3), match='ea'>
None
<re.Match object; span=(3, 5), match='71'>


**Greedines**

With operators `*`, `+` the resulting action is to consume as much of the pattern as possible/will match with the longest string they can find. That is

In [109]:
# Pattern strarting with < followed by any number of non-newline charancters followed by >
print(re.search('<.*>', '<html><head><title>Title</title>'))
#                        |start                         |stop     in greedy way

<re.Match object; span=(0, 32), match='<html><head><title>Title</title>'>


In [110]:
print(re.search('<.*?>', '<html><head><title>Title</title>'))
#                        |start|stop     in non-greedy way stop at first match

<re.Match object; span=(0, 6), match='<html>'>


## Grouping and capturing

Placing parts of the regex inside `()` will group that part of the regex together. Let's look for groups of digits

In [124]:
# Say you have a phone number of the for XXX-YYYY
match = re.search('(\d{3})-(\d{4})', '123-3333')

In [128]:
print(match.groups())
print(match.group(0))  # Entire match
print(match.group(1))  # Get the group by order (staring from 1)
print(match.group(2))

('123', '3333')
123-3333
123
3333


We can use reffer back to captured groups to build up patterns

In [132]:
# The first match shold be repeated
print(re.search(r'(\d{3})-(\d{4})-\1', '123-3333-444'))
print(re.search(r'(\d{3})-(\d{4})-\1', '123-3333-123'))

None
<re.Match object; span=(0, 12), match='123-3333-123'>


With many groups it might be convenient to name them, define `(?P<name>)`, refer `(?P=name)` 

In [209]:
match = re.search('(?P<area>\d{3})-(?P<extension>\d{4})', '123-3333')
print(match.group('area'), match.group('extension'))

match = re.search('(?P<area>\d{3})-(?P<extension>\d{4})-(?P=area)', '123-3333-123')
print(match)
print(match.groups())

123 3333
<re.Match object; span=(0, 12), match='123-3333-123'>
('123', '3333')


Sometimes we might not want to capture/skip the group, `(?:...)`

In [154]:
match = re.search('(?P<area>\d{3})-(?:\d{4})-(?P=area)', '123-3333-123')
print(match)
print(match.groups())

<re.Match object; span=(0, 12), match='123-3333-123'>
('123',)


## Lookahead and Lookbehind assertions

`(?=<lookahead_regex>)` asserts that what follows the regex parser’s current position must match `<lookahead_regex>`

In [214]:
# Match pattern of 3 digits if they are followed by 4 letter string
pattern = '\d{3}-(?=[a-zA-Z]{4})'
print(re.search(pattern, '123-4568'))

match = re.search(pattern, '123-miro')
print(match)
print(match.groups())

None
<re.Match object; span=(0, 4), match='123-'>
()


In [215]:
# We can again name groups
match = re.search('(\d{3})-(?=(?P<name>[a-zA-Z]{4}))', '123-miro')
print(match)
match.group(1, 'name')

<re.Match object; span=(0, 4), match='123-'>


('123', 'miro')

`(?<=<lookbehind_regex>)` asserts that what precedes the regex parser’s current position must match `<lookbehind_regex>`

In [217]:
# Match the 3 digits if they are preceed by string of 4 letters
pattern = '(?<=(?P<name>^[a-zA-Z]{4})):(?P<phone>\d{3})'
match = re.search(pattern, 'miroslav:123')
print(match)  # Note that we are getting the match on the digits

match = re.search(pattern, 'miro:123')
print(match)
match.group('name', 'phone')

None
<re.Match object; span=(4, 8), match=':123'>


('miro', '123')

## Verbose patterns
Regular expression can become long and convoluted in which case it comes in handy to be able to document them. To this end we can use the `re.VERBOSE` flag. Suppose we want to parse phone numbers of the form `(area code) 4digits 4digits`

In [157]:
pattern = r'\((\d{3})\)\D*(\d{4})\D*(\d{4})'
print(re.search(pattern, '800-5555-1234'))
print(re.search(pattern, '(800)-5555-1234'))

And now a  documented version

In [218]:
pattern = r'''
\(         # Area code begins with (
(\d{3})    # is followed by 3 digits
\)         # and close area code
\D*        # There can be some no digits spaces until the next group
(\d{4})    # of four digits
\D*        # Again some separators
(\d{4})    # and finally a 4 digit group
'''
match = re.search(pattern, '(800)- - 5555--1234', re.VERBOSE)
print(match)
print(match.groups)

<re.Match object; span=(0, 19), match='(800)- - 5555--1234'>
<built-in method groups of re.Match object at 0x7fa6730e7350>


## Functions of the `re` module
`re.search` stopped once we found a first match. All matches can be found with `re.findall`

In [162]:
string = 'a=1 aa=2 b c aaa=3'
print(re.findall('a=\d+', string))
# Contrast with the following patterns where we introduce 2 groups
print(re.findall('(a)=(\d+)', string))

['a=1', 'a=2', 'a=3']
[('a', '1'), ('a', '2'), ('a', '3')]


`re.split` lets us split the string where the pattern matched. In the following example we consider numbers whose digits form repeating patterns. These patterns are our split points.

In [219]:
import sympy as sp

r = sp.Rational(1, 7)
n = str(sp.N(r, 50))
print(n)  # See that they repeat


pattern = r'(\d+?)\1'
re.search(pattern, n).groups()

re.split(pattern, n)

0.14285714285714285714285714285714285714285714285714


['0.', '142857', '', '142857', '', '142857', '', '142857', '14']

`re.sub` will perform substitution for the matched substring. A cool feature is that we can use the groups to build the replacent string. In the following we assume string in the form `DAY MONTH YEAR` encoded respectively as 3-letter word and strings of 2 and 4 digits. We are only after the year and month in the replaced string.

In [222]:
pattern = r'([A-Z]{3}) (?:\d{2}) (\d{4})'
string = 'MON 12 2022'
re.search(pattern, string)
re.sub(pattern, r'replaced: \2 \1', string)

'replaced: 2022 MON'

So far, we have used top-level functions. A building block of these is a pattern which can be complied by `re.compile`.
Citing the [re documentation](https://docs.python.org/3/howto/regex.html): _Under the hood, these functions simply create a pattern object for you and call the appropriate method on it. They also store the compiled object in a cache, so future calls using the same RE won’t need to parse the pattern again and again._

_Should you use these module-level functions, or should you get the pattern and call its methods yourself? If you’re accessing a regex within a loop, pre-compiling it will save a few function calls. Outside of loops, there’s not much difference thanks to the internal cache._

__Question__: 1. Verify the above by profiling.

# Case studies with RegEx

In [ ]:

Questions: 
- How do you match the "4"?
- How do you match everything except the "4"?
- How do you match the *first* "hat"?
- How many regexes can you make that matches "ccc" in "abccc"?

- Take as your test string: `Norwegian: "God dag". Italian: "Buongiorno"`
Can you match the greetings?


***


Question: 
- Can you make a regex that matches any character repeated twice?
For example, your regex should match "ee" in "Rita Skeeter".
- Can you make a regex that matches any repeated character?
For example, your regex should match "sss" in "headmistressship".
- Bonus: If you work on a text for a long time, you eventually become completely blind to your own mistakes. For example, my master thesis contains double "the the" 3 times. Can you make a regex that finds repeated words?

That concludes basic regex syntax! Now onto python and regex. 

## Additional Resources ##

Videos from Simon Funke from previous editions of the course

- [Regex 1](https://www.youtube.com/watch?v=ma93hpNFXZM)
- [Regex 2](https://www.youtube.com/watch?v=B6XoKtQA2Fc)
- [Regex 3](https://www.youtube.com/watch?v=jxNLY0L_N78)
- [Regex 4](https://www.youtube.com/watch?v=j1jW5EF5jfs)

- RegEx [golf](https://nbviewer.org/url/norvig.com/ipython/xkcd1313.ipynb) by Google's Peter Norvig.
- Nice [tutorial](https://realpython.com/regex-python/) on RegEx in Python 